<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Usando la API para obtener datos sobre personas desaparecidas del RNPDNO


<p> Julio Waissman Vilanova </p>



<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/RNPDNO-API.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

In [2]:
import os
import sys
import requests
import datetime

import pandas as pd
import json


## Calentando motores

Para descargar los datos, vamos a consultar directamente la base de datos pública del [Registro Nacional de Personas Desaparecidas y No Localizadas (RNPDNO)](https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index).

El RNPDNO no tiene una API tal cual, sin embargo, [Pablo Reyes Moctezuma](https://github.com/pablorm296) encontró una manera de extraer la información usando la librería `request` de python. La API que, me imagino, el extrajo a punta de prueba y error la documento en [este archivo en markdown](https://github.com/pablorm296/ScrapperRNPDNO/blob/master/Test/API.md). Un chambón.

Vamos air la usando poco a poco, empecemos por tratar de encontrar en el catálogo los indices de estados, municipios y colonias.

In [3]:
API_HOST = "https://versionpublicarnpdno.segob.gob.mx/"
API_SOCIODEOGRAFICOS_ROOT = "Sociodemografico/"
API_CATALAGO_ROOT = "Catalogo/"

ENDPOINT_CATALOGO_EDO = "Estados/"
ENDPOINT_CATALOGO_MUN = "Municipios/"
ENDPOINT_CATALOGO_COL = "Colonias/"

# Before doing anything, we must make a dummy request to the index in order to get the propper cookies
main_session = requests.Session()
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index")
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/ContextoGeneral")

<Response [200]>

Los identificadores de los estados:

In [4]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_EDO

r = main_session.post(TARGET_URL)
estados_id = pd.json_normalize(r.json(),)
estados_id.columns = ['Valor', 'Estado']
estados_id

,Valor,Estado
0,0,--TODOS--
1,1,AGUASCALIENTES
2,2,BAJA CALIFORNIA
3,3,BAJA CALIFORNIA SUR
4,4,CAMPECHE
5,7,CHIAPAS
6,8,CHIHUAHUA
7,9,CIUDAD DE MEXICO
8,5,COAHUILA
9,6,COLIMA


y ahora los municipios de Sonora

In [5]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_MUN
DATA = {"idEstado": "26"}

r = main_session.post(TARGET_URL, data = DATA)
mun_son_id = pd.json_normalize(r.json())
mun_son_id.columns = ['Valor', 'Municipio']
mun_son_id

,Valor,Municipio
0,0,--TODOS--
1,1,ACONCHI
2,2,AGUA PRIETA
3,3,ALAMOS
4,4,ALTAR
...,...,...
69,65,TUBUTAMA
70,66,URES
71,67,VILLA HIDALGO
72,68,VILLA PESQUEIRA


y por último los identificadores de las colonias del municipio de Hermosillo

In [6]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_COL
DATA = {"idEstado": "26", "idMunicipio": "30"}

r = main_session.post(TARGET_URL, data = DATA)
col_hmo_id = pd.json_normalize(r.json())
col_hmo_id.columns = ['Valor', 'Municipio']
col_hmo_id

,Valor,Municipio
0,0,--TODAS--
1,347025,22 DE SEPTIEMBRE
2,347004,26 DE OCTUBRE
3,347026,4 DE MARZO
4,347027,4 OLIVOS
...,...,...
631,347358,VISTA DEL LAGO
632,347359,Y
633,347582,ZACATON
634,347384,ZAMORA


## Sociodemográficos totales

`Con este `endpoint` se pueden consultar resúmenes generales de la información que se pide. recuerda de revisar los catálogos.

Hay dos variables cuyos valores posibles son los siguientes:

**idEstatusVictima**:
- "0" PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS
- "2" PERSONAS LOCALIZADAS CON VIDA
- "3" PERSONAS LOCALIZADAS SIN VIDA- "4" PERSONAS DESAPARECIDAS
- "5" PERSONAS NO LOCALIZADAS
- "6" PERSONAS LOCALIZADAS
- "7" PERSONAS DESAPARECIDAS Y NO LOCALIZADAS

**idHipotesisNoLocalizacion**:
- "0" --TODAS--
- "1" ACCIDENTE
- "2" CATÁSTROFE
- "3" NO LOCALIZACIÓN VOLUNTARIA
- "4" NO LOCALIZACIÓN INVOLUNTARIA
- "5" SE DESCONOCE

Veamos como funciona pidiendo información de Sonora y de Hermosillo. Empecemos por Sonora

In [194]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "Totales"

DATA = {
  "titulo":"",
  "subtitulo": "",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
  "idDelito":"0"
}
r = main_session.post(TARGET_URL, json = DATA)

resumen_sonora = pd.json_normalize(r.json()).T
resumen_sonora.columns = ['Valor']

resumen_sonora


,Valor
TotalGlobal,"7,960"
TotalDesaparecidos,"4,770"
TotalLocalizados,"3,190"
PorcentajeDesaparecidos,59.92 %
PorcentajeLocalizados,40.08 %
TotalSoloDesaparecidos,"4,730"
TotalSoloNoLocalizados,40
PorcentajeSoloDesaparecidos,99.16 %
PorcentajeSoloNoLocalizados,0.84 %
TotalLocalizadosCV,"2,929"


### Ejercicio

Probar con diferentes consultas y tratar de inferir los valores que pueden tomar (o buscarlas en la documentación de la API) las diferentes variables que pueden servir para encontrar búsquedas más específicas.

Por ejemplo, ¿Como podríamos consultar las estadísticas sobre mujeres desaparecidas en el municipio de Cajeme?

In [189]:
cajeme_id = mun_son_id.loc[mun_son_id['Municipio'] == 'CAJEME', 'Valor'].values[0]
cajeme_id

18

In [195]:
# Estadísticas sobre mujeres desaparecidas en el municipio de Cajeme por colonia

TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR COLONIAS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":str(cajeme_id),
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)
res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Colonia'] = res['XAxisCategories']
datos_cajeme = pd.DataFrame(datos)
datos_cajeme.columns = ['Hombre', 'Mujer', 'Indeterminado', 'Colonia']
datos_cajeme[['Mujer', 'Colonia']]

,Mujer,Colonia
0,1,EJIDAL
1,1,AMANECER 1
2,2,VILLAS DEL REAL
3,1,VALLE DORADO
4,0,MONTECARLOS
5,0,NOROESTE
6,1,LA CORTINA
7,2,VILLA BONITA
8,4,CIUDAD OBREGÓN
9,0,PARQUE INDUSTRIAL


## Personas desaparecidas por sexo y colonia

El `endpoint` **BarChartSexoColonia** está diseñado para generar gráficas, pero nos permite extraer información, si la sabemos formatear.

Vamos viendo un ejemplo:


In [77]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR COLONIAS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":"30",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)

res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Colonia'] = res['XAxisCategories']
por_colonia = pd.DataFrame(datos)
por_colonia.index = por_colonia.Colonia

por_colonia


,Hombre,Mujer,Indeterminado,Colonia
Colonia,,,,
VILLA HERMOSA,0,4,0,VILLA HERMOSA
MIGUEL ALEMÁN CENTRO,11,6,0,MIGUEL ALEMÁN CENTRO
QUINTA EMILIA,2,0,0,QUINTA EMILIA
ARBOLEDAS,1,1,0,ARBOLEDAS
SAHUARO INDECO,0,2,0,SAHUARO INDECO
UNIVERSIDAD,1,0,0,UNIVERSIDAD
JESÚS GARCIA,7,3,0,JESÚS GARCIA
MESA DEL SERI,1,1,0,MESA DEL SERI
NUEVA PALMIRA,1,0,0,NUEVA PALMIRA


### Ejercicio

¿Como podemos sacar lo que pasa en todo el estado, por municipios y por colonias? Intentalo.

In [196]:
# Personas desaparecidas por sexo en todo el estado

TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoMunicipio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR MUNICIPIOS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)

res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Municipio'] = res['XAxisCategories']
por_municipio = pd.DataFrame(datos)

total_estado = por_municipio[['Hombre', 'Mujer', 'Indeterminado']].sum()
df_estado = pd.DataFrame(total_estado)
df_estado.columns=['Total']
df_estado

,Total
Hombre,5321
Mujer,2479
Indeterminado,6


In [197]:
# Personas desaparecidas por sexo por municipios

TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoMunicipio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR MUNICIPIOS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)

res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Municipio'] = res['XAxisCategories']
por_municipio = pd.DataFrame(datos)
por_municipio.index = por_municipio.Municipio

por_municipio

,Hombre,Mujer,Indeterminado,Municipio
Municipio,,,,
HERMOSILLO,1231,722,0,HERMOSILLO
NOGALES,669,403,0,NOGALES
CAJEME,649,317,0,CAJEME
GUAYMAS,331,93,0,GUAYMAS
AGUA PRIETA,320,182,0,AGUA PRIETA
CABORCA,286,83,0,CABORCA
PUERTO PEÑASCO,248,109,0,PUERTO PEÑASCO
SAN LUIS RÍO COLORADO,225,168,1,SAN LUIS RÍO COLORADO
NAVOJOA,177,64,0,NAVOJOA


In [198]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_MUN
DATA = {"idEstado": "26"}

r = main_session.post(TARGET_URL, data = DATA)
mun_son_id = pd.json_normalize(r.json())
mun_son_id.columns = ['Valor', 'Municipio']
mun_son_ids = mun_son_id.values[:,0]
mun_son_ids

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 71, 16, 17,
       18, 19, 20, 22, 23, 24, 25, 26, 27, 70, 28, 29, 30, 31, 32, 33, 34,
       35, 21, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 72, 54, 55, 56, 57, 58, 59, 60, 73, 61, 62, 63, 64, 65,
       66, 67, 68, 69], dtype=object)

In [199]:
# Personas desaparecidas por sexo por colonias por municipios

TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

df_col = pd.DataFrame()

for i in range(1, len(mun_son_ids)):
  DATA = {
    "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
    "subtitulo":"POR COLONIAS - HERMOSILLO",
    "idEstado":"26",
    "idMunicipio": i,
    "idColonia":"0",
    "idEstatusVictima":"0",
    "idHipotesisNoLocalizacion":"0",
    "idDelito":"0",
    "fechaInicio":"",
    "fechaFin":"",
    "mostrarFechaNula":"0",
    "idNacionalidad":"0",
    "edadInicio":"",
    "edadFin":"",
    "mostrarEdadNula":"0",
    "idHipotesis":"",
    "idMedioConocimiento":"",
    "idCircunstancia":"",
    "tieneDiscapacidad":"",
    "idTipoDiscapacidad":"0",
    "idEtnia":"0",
    "idLengua":"0",
    "idReligion":"",
    "esMigrante":"",
    "idEstatusMigratorio":"0",
    "esLgbttti":"",
    "esServidorPublico":"",
    "esDefensorDH":"",
    "esPeriodista":"",
    "esSindicalista":"",
    "esONG":"",
  }
  r = main_session.post(TARGET_URL, json = DATA)

  res = r.json()
  datos = {serie['name']: serie['data'] for serie in res['Series']}
  datos['Colonia'] = res['XAxisCategories']

  por_colonia = pd.DataFrame(datos)
  por_colonia['Municipio'] = mun_son_id.loc[mun_son_id['Valor'] == i].values[0][1]
  por_colonia.index = por_colonia.Colonia

  por_colonia
  df_col = pd.concat([df_col, por_colonia], axis=0, ignore_index=True)

df_col

,Hombre,Mujer,Indeterminado,Colonia,Municipio
0,2.0,0.0,0.0,ACONCHI,ACONCHI
1,2.0,3.0,0.0,SIN COLONIA DE REFERENCIA,ACONCHI
2,0.0,3.0,0.0,VALLES DUARTE,AGUA PRIETA
3,4.0,6.0,0.0,BUENOS AIRES,AGUA PRIETA
4,3.0,3.0,0.0,PROGRESO,AGUA PRIETA
...,...,...,...,...,...
421,1.0,0.0,0.0,15 DE MAYO,BENITO JUÁREZ
422,1.0,0.0,0.0,ROSENDO MONTAÑO,SAN IGNACIO RÍO MUERTO
423,1.0,0.0,0.0,SAN IGNACIO RIO MUERTO,SAN IGNACIO RÍO MUERTO
424,14.0,5.0,0.0,SIN COLONIA DE REFERENCIA,SAN IGNACIO RÍO MUERTO


## Información por sexo y por año

Tambien se puede encontrar información por sexo y por año utilizando otro `endpoint`: **AreaChartSexoAnio**

Sin mas choro, vamos a ver como se usa, otra vez con el estado de Sonora:

In [98]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR AÑO EN EL ESTADO DE SONORA",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "idEstatusVictima":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}

r = main_session.post(TARGET_URL, json = DATA)

res = r.json()

datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Fecha'] = res['XAxisCategories']

por_fecha = pd.DataFrame(datos)
por_fecha['Fecha'] = pd.to_numeric(por_fecha.Fecha, errors='coerce')
por_fecha.index = por_fecha.Fecha

por_fecha


,Hombre,Mujer,Indeterminado,Fecha
Fecha,,,,
2007,0,1,0,2007
2011,1,0,0,2011
2015,0,1,0,2015
2017,1,0,0,2017
2019,1,0,0,2019
2020,2,0,0,2020


### Ejercicio

¿Se puede hacer por municipio? ¿En forma programática? ¿Para algun caso especial? Intentalo

In [200]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

df_fecha = pd.DataFrame()

for i in range(1, len(mun_son_ids)):
  DATA = {
    "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
    "subtitulo":"POR AÑO EN EL ESTADO DE SONORA",
    "idEstado":"26",
    "idMunicipio": i,
    "idColonia":"0",
    "edadInicio":"",
    "edadFin":"",
    "mostrarEdadNula":"0",
    "idHipotesisNoLocalizacion":"0",
    "idDelito":"0",
    "idEstatusVictima":"0",
    "fechaInicio":"",
    "fechaFin":"",
    "mostrarFechaNula":"0",
    "idNacionalidad":"0",
    "idHipotesis":"",
    "idMedioConocimiento":"",
    "idCircunstancia":"",
    "idEtnia":"0",
    "idLengua":"0",
    "idReligion":"",
    "tieneDiscapacidad":"",
    "idTipoDiscapacidad":"0",
    "esMigrante":"",
    "idEstatusMigratorio":"0",
    "esLgbttti":"",
    "esServidorPublico":"",
    "esDefensorDH":"",
    "esPeriodista":"",
    "esSindicalista":"",
    "esONG":"",
  }

  r = main_session.post(TARGET_URL, json = DATA)

  res = r.json()

  datos = {serie['name']: serie['data'] for serie in res['Series']}
  datos['Fecha'] = res['XAxisCategories']

  por_fecha = pd.DataFrame(datos)
  por_fecha['Fecha'] = pd.to_numeric(por_fecha.Fecha, errors='coerce')
  # por_fecha.index = por_fecha.Fecha

  por_fecha['Municipio'] = mun_son_id.loc[mun_son_id['Valor'] == i].values[0][1]
  # por_colonia.index = por_colonia.Colonia

  df_fecha = pd.concat([df_fecha, por_fecha], axis=0, ignore_index=True)

df_fecha['Fecha'] = df_fecha['Fecha'].astype('Int64')
df_fecha

,Hombre,Mujer,Indeterminado,Fecha,Municipio
0,1.0,0.0,0.0,<NA>,ACONCHI
1,0.0,1.0,0.0,2013,ACONCHI
2,1.0,0.0,0.0,2016,ACONCHI
3,2.0,0.0,0.0,2017,ACONCHI
4,0.0,1.0,0.0,2019,ACONCHI
...,...,...,...,...,...
557,3.0,0.0,0.0,2020,SE DESCONOCE
558,16.0,1.0,0.0,2021,SE DESCONOCE
559,9.0,0.0,0.0,2022,SE DESCONOCE
560,5.0,2.0,0.0,2023,SE DESCONOCE


### Ejercicio

Extrae alguna información del conjunto de tados que pienses que es relevante, y explica porqué.

In [167]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "Totales"
DATA_DESAPARECIDOS = {
  "titulo":"",
  "subtitulo": "",
  "idEstatusVictima":"7",
  "idHipotesisNoLocalizacion":"",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
  "idDelito":"0"
}
r = main_session.post(TARGET_URL, json = DATA_DESAPARECIDOS)
total_desaparecidos = pd.json_normalize(r.json())['TotalGlobal'].values[0]
total_desaparecidos

'4,770'

In [174]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "Totales"
DATA_CON_VIDA = {
  "titulo":"",
  "subtitulo": "0",
  "idEstatusVictima":"2",
  "idHipotesisNoLocalizacion":"0",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
  "idDelito":"0"
}
r = main_session.post(TARGET_URL, json = DATA_CON_VIDA)
total_con_vida = pd.json_normalize(r.json())['TotalGlobal'].values[0]
total_con_vida

'2,929'

In [176]:
# Por ciento de desaparecidos encontrados con vida

if isinstance(total_desaparecidos, str):
    total_desaparecidos = int(total_desaparecidos.replace(',', ''))
if isinstance(total_con_vida, str):
    total_con_vida = int(total_con_vida.replace(',', ''))

con_vida = round(total_con_vida / total_desaparecidos * 100, 2)
print(f'El {con_vida}% por ciento de los desaparecidos en Sonora se encontraron con vida')

El 61.4% por ciento de los desaparecidos en Sonora se encontraron con vida


## Practicando a ser investigador de APIs

Ahora te pido que revises si puedes encontrar otros endpoints para recuperar mas información de las bases que no se encuentran liberadas. Puede ser en la misma página, o en blogs o revisando código. Agrega en esta libreta la documentación (o enlaces a dicha documentación) y un ejemplo de uso de una API pobremente documentada.